In [11]:
"""
BOPTEST HVAC DQN 训练 - 修复版（移除不必要的 select 步骤）
"""

import os
import time
from datetime import datetime
from collections import Counter

import numpy as np
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# =====================================================================
# 配置参数
# =====================================================================
BOPTEST_URL = os.getenv("BOPTEST_URL", "http://localhost:80")
TESTCASE_NAME = "bestest_air"

TOTAL_EPISODES = 500
STEPS_PER_EPISODE = 96*5
EVAL_FREQUENCY = 20
PLOT_SAVE_DIR = "training_output"

# DQN 超参数
GAMMA = 0.99
LR = 1e-4
BATCH_SIZE = 64*5
BUFFER_SIZE = 50_000*5
MIN_REPLAY_SIZE = 1_000*5
TARGET_UPDATE = 500
EPS_START = 1.0
EPS_END = 0.05
EPS_DECAY_STEPS = 15_000
TAU = 0.01

# 动作空间
FAN_LEVELS = [0.3, 0.6, 0.9]
SUPPLY_TEMP_LEVELS = [288.15, 296.15]
HEAT_SETPOINT_LEVELS = [294.15, 296.15]
COOL_SETPOINT_LEVELS = [299.15, 301.15]

NUM_ACTIONS = (
    len(FAN_LEVELS)
    * len(SUPPLY_TEMP_LEVELS)
    * len(HEAT_SETPOINT_LEVELS)
    * len(COOL_SETPOINT_LEVELS)
)

# =====================================================================
# 工具函数
# =====================================================================

def log(msg, level="INFO"):
    ts = datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {level:5s} | {msg}", flush=True)


def safe_json(resp, ctx=""):
    try:
        return resp.json()
    except Exception as e:
        log(
            f"JSON 解析失败({ctx}): {type(e).__name__}: {e}. status={resp.status_code}, text={resp.text[:300]}",
            "ERROR",
        )
        return None


def K2C(k):
    return k - 273.15


def action_to_values(action_idx):
    cool_idx = action_idx % len(COOL_SETPOINT_LEVELS)
    remaining = action_idx // len(COOL_SETPOINT_LEVELS)
    heat_idx = remaining % len(HEAT_SETPOINT_LEVELS)
    remaining //= len(HEAT_SETPOINT_LEVELS)
    supply_idx = remaining % len(SUPPLY_TEMP_LEVELS)
    fan_idx = remaining // len(SUPPLY_TEMP_LEVELS)

    v = {
        "fan": FAN_LEVELS[fan_idx],
        "supply_temp": SUPPLY_TEMP_LEVELS[supply_idx],
        "heat_setpoint": HEAT_SETPOINT_LEVELS[heat_idx],
        "cool_setpoint": COOL_SETPOINT_LEVELS[cool_idx],
    }

    if v["cool_setpoint"] <= v["heat_setpoint"] + 1.5:
        v["cool_setpoint"] = v["heat_setpoint"] + 1.5

    v["fan"] = float(np.clip(v["fan"], 0.0, 1.0))
    v["supply_temp"] = float(np.clip(v["supply_temp"], 285.15, 313.15))
    v["heat_setpoint"] = float(np.clip(v["heat_setpoint"], 278.15, 308.15))
    v["cool_setpoint"] = float(np.clip(v["cool_setpoint"], 278.15, 308.15))
    return v


def action_to_string(action_idx):
    v = action_to_values(action_idx)
    return (
        f"Fan={v['fan']:.1f}|供风={K2C(v['supply_temp']):.0f}°C|"
        f"热设={K2C(v['heat_setpoint']):.1f}°C|冷设={K2C(v['cool_setpoint']):.1f}°C"
    )

# =====================================================================
# 环境封装（修复版 - 直接使用 testcase 名称）
# =====================================================================

class BOPTESTEnv:
    def __init__(self):
        self.url = BOPTEST_URL
        self.testid = None  # 需要通过 select 获取
        self.obs_dim = 12
        self.action_dim = NUM_ACTIONS
        self.prev_action_norm = 0.0

    def _select_testcase(self):
        """选择测试案例，获取 testid"""
        log(f"选择测试案例: {TESTCASE_NAME}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                log(f"尝试选择测试案例 (尝试 {attempt + 1}/{max_retries})...")
                resp = requests.post(
                    f"{self.url}/testcases/{TESTCASE_NAME}/select", 
                    timeout=120
                )
                if resp.status_code == 200:
                    data = safe_json(resp, "select testcase")
                    if data:
                        self.testid = data.get('testid') or data.get('testcaseid')
                        if self.testid:
                            log(f"✅ Test ID: {self.testid}")
                            time.sleep(3)  # 等待 worker 初始化
                            return True
                        else:
                            log("❌ 未返回 testid", "ERROR")
                else:
                    log(
                        f"❌ 选择失败，状态码={resp.status_code}, 响应={resp.text[:200]}",
                        "ERROR",
                    )
            except requests.exceptions.Timeout:
                log(f"❌ 选择超时 (尝试 {attempt + 1}/{max_retries})", "WARN")
                if attempt < max_retries - 1:
                    wait_time = 10 * (attempt + 1)
                    log(f"等待 {wait_time} 秒后重试...")
                    time.sleep(wait_time)
            except Exception as e:
                log(f"❌ 选择失败: {e} (尝试 {attempt + 1}/{max_retries})", "ERROR")
                if attempt < max_retries - 1:
                    wait_time = 10 * (attempt + 1)
                    log(f"等待 {wait_time} 秒后重试...")
                    time.sleep(wait_time)
        return False

    def reset(self):
        """重置环境"""
        log("开始重置环境...")
        
        # 每次 reset 都重新选择测试案例（确保获取新的 testid）
        # 如果之前有 testid，先停止它
        if self.testid:
            try:
                requests.put(f"{self.url}/stop/{self.testid}", timeout=10)
                log(f"已停止之前的 testid: {self.testid}")
            except:
                pass
            self.testid = None
        
        # 选择测试案例，获取新的 testid
        if not self._select_testcase():
            raise Exception("无法选择测试案例")
        
        try:
            # 步骤1: 初始化
            log("  → Initialize...")
            resp = requests.put(
                f"{self.url}/initialize/{self.testid}",
                json={"start_time": 0, "warmup_period": 0},
                timeout=120,
            )
            if resp.status_code != 200:
                raise Exception(f"Initialize 失败: {resp.status_code}, {resp.text[:200]}")
            log("  ✓ Initialize 成功")
            
            # 步骤2: 设置步长
            log("  → Set step...")
            resp = requests.put(
                f"{self.url}/step/{self.testid}", 
                json={"step": 900}, 
                timeout=60,
            )
            if resp.status_code != 200:
                raise Exception(f"Set step 失败: {resp.status_code}")
            log("  ✓ Set step 成功")
            
            # 步骤3: 第一次 advance
            log("  → Advance...")
            resp = requests.post(
                f"{self.url}/advance/{self.testid}", 
                json={}, 
                timeout=60,
            )
            if resp.status_code != 200:
                raise Exception(f"Advance 失败: {resp.status_code}")
            
            payload = resp.json().get('payload', {})
            self.prev_action_norm = 0.0
            log("✅ 环境重置成功")
            return self._get_obs(payload)
            
        except Exception as e:
            log(f"❌ 重置失败: {e}", "ERROR")
            raise

    def step(self, action_idx):
        action = action_to_values(action_idx)
        control = {
            'fcu_oveFan_u': action['fan'],
            'fcu_oveFan_activate': 1,
            'fcu_oveTSup_u': action['supply_temp'],
            'fcu_oveTSup_activate': 1,
            'con_oveTSetHea_u': action['heat_setpoint'],
            'con_oveTSetHea_activate': 1,
            'con_oveTSetCoo_u': action['cool_setpoint'],
            'con_oveTSetCoo_activate': 1,
        }

        resp = requests.post(
            f"{self.url}/advance/{self.testid}", json=control, timeout=60
        )
        payload = resp.json().get('payload', {}) or {}

        obs = self._get_obs(payload)
        reward, reward_detail = self._calc_reward(obs)

        sim_time = payload.get("time", 0)
        done = sim_time >= STEPS_PER_EPISODE * 900

        info = {
            "sim_time": sim_time,
            "room_temp": K2C(obs[0]),
            "outdoor_temp": K2C(obs[1]),
            "reward_detail": reward_detail,
        }
        self.prev_action_norm = action_idx / max(1, NUM_ACTIONS - 1)
        return obs, reward, done, info

    def _safe_get(self, payload, key, default):
        v = payload.get(key, default)
        if v is None:
            return default
        return v

    def _get_obs(self, payload):
        if not payload:
            return np.zeros(self.obs_dim, dtype=np.float32)

        sim_time = payload.get("time", 0)
        hour = (sim_time % 86400) / 3600.0

        room_temp = self._safe_get(payload, "zon_reaTRooAir_y", 293.15)
        outdoor_temp = self._safe_get(payload, "zon_weaSta_reaWeaTDryBul_y", 283.15)
        rel_hum = self._safe_get(payload, "zon_weaSta_reaWeaRelHum_y", 0.5)
        solar = self._safe_get(payload, "zon_weaSta_reaWeaHGloHor_y", 0.0)
        wind = self._safe_get(payload, "zon_weaSta_reaWeaWinSpe_y", 0.0)
        co2 = self._safe_get(payload, "zon_reaCO2RooAir_y", 600.0)
        p_heating = self._safe_get(payload, "fcu_reaPHea_y", 0.0)
        p_cooling = self._safe_get(payload, "fcu_reaPCoo_y", 0.0)
        p_fan = self._safe_get(payload, "fcu_reaPFan_y", 0.0)

        obs = np.array(
            [
                room_temp,
                outdoor_temp,
                rel_hum,
                solar / 1000.0,
                wind,
                co2 / 1000.0,
                p_heating / 1000.0,
                p_cooling / 1000.0,
                p_fan / 1000.0,
                np.sin(2 * np.pi * hour / 24),
                np.cos(2 * np.pi * hour / 24),
                self.prev_action_norm,
            ],
            dtype=np.float32,
        )
        return obs

    def _calc_reward(self, obs):
        room_temp_c = K2C(obs[0])
        p_h, p_c, p_f = obs[6], obs[7], obs[8]

        temp_dev = max(0.0, abs(room_temp_c - 22.0) - 1.0)
        comfort = -1.0 * (temp_dev ** 2)
        energy = -0.5 * (p_h + p_c) - 0.1 * p_f
        smooth = -0.05 * abs(obs[-1] - self.prev_action_norm)

        reward = comfort + energy + smooth
        detail = {
            "comfort": comfort,
            "energy": energy,
            "smooth": smooth,
            "room_temp_c": room_temp_c,
        }
        return reward, detail

    def stop(self):
        if self.testid:
            try:
                requests.put(f"{self.url}/stop/{self.testid}", timeout=10)
                log(f"✅ 环境已停止 (testid: {self.testid})")
            except Exception as e:
                log(f"停止失败: {e}", "WARN")
            finally:
                self.testid = None  # 清空 testid，下次 reset 时会重新获取

# =====================================================================
# DQN 组件
# =====================================================================

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, action_dim),
        )

    def forward(self, x):
        return self.net(x)


class ReplayBuffer:
    def __init__(self, capacity, obs_dim):
        self.capacity = capacity
        self.obs = np.zeros((capacity, obs_dim), dtype=np.float32)
        self.next_obs = np.zeros((capacity, obs_dim), dtype=np.float32)
        self.actions = np.zeros(capacity, dtype=np.int64)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=np.float32)
        self.idx = 0
        self.full = False

    def add(self, obs, action, reward, done, next_obs):
        self.obs[self.idx] = obs
        self.next_obs[self.idx] = next_obs
        self.actions[self.idx] = action
        self.rewards[self.idx] = reward
        self.dones[self.idx] = done
        self.idx = (self.idx + 1) % self.capacity
        if self.idx == 0:
            self.full = True

    def sample(self, batch_size):
        max_idx = self.capacity if self.full else self.idx
        idxs = np.random.choice(max_idx, batch_size, replace=False)
        batch = (
            torch.FloatTensor(self.obs[idxs]),
            torch.LongTensor(self.actions[idxs]),
            torch.FloatTensor(self.rewards[idxs]),
            torch.FloatTensor(self.dones[idxs]),
            torch.FloatTensor(self.next_obs[idxs]),
        )
        return batch


# =====================================================================
# 监控与可视化
# =====================================================================

class Monitor:
    def __init__(self):
        self.train_rewards = []
        self.train_temps = []
        self.eval_rewards = []
        self.eval_episodes = []
        self.last_actions = []
        self.last_temps = []
        self.last_outdoor = []
        self.last_rewards = []
        self.last_action_counts = Counter()

    def log_episode_curves(self, actions, temps, outdoors, rewards):
        self.last_actions = actions
        self.last_temps = temps
        self.last_outdoor = outdoors
        self.last_rewards = rewards
        self.last_action_counts = Counter(actions)

    def plot(self):
        fig, axes = plt.subplots(1, 3, figsize=(12, 3))

        ax = axes[0]
        ax.plot(self.train_rewards, 'b-', alpha=0.6)
        if len(self.train_rewards) >= 5:
            ma = np.convolve(self.train_rewards, np.ones(5) / 5, mode='valid')
            ax.plot(range(4, len(self.train_rewards)), ma, 'r-', linewidth=2)
        ax.set_title('Train Reward')
        ax.set_xlabel('Episode')
        ax.grid(True, alpha=0.3)

        ax = axes[1]
        ax.plot(self.train_temps, 'g-')
        ax.axhspan(20, 24, alpha=0.1, color='green')
        ax.axhline(22, color='r', linestyle='--')
        ax.set_title('Avg Temp (°C)')
        ax.set_xlabel('Episode')
        ax.grid(True, alpha=0.3)

        ax = axes[2]
        if self.eval_rewards:
            ax.plot(self.eval_episodes, self.eval_rewards, 'ro-')
        ax.set_title('Eval Reward')
        ax.set_xlabel('Episode')
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def plot_episode_curves(self, episode, save_dir=None):
        if not self.last_actions:
            return
        steps = range(1, len(self.last_actions) + 1)

        decoded = [action_to_values(a) for a in self.last_actions]
        fan_seq = [d['fan'] for d in decoded]
        supply_seq = [K2C(d['supply_temp']) for d in decoded]
        heat_seq = [K2C(d['heat_setpoint']) for d in decoded]
        cool_seq = [K2C(d['cool_setpoint']) for d in decoded]

        fig, axes = plt.subplots(3, 2, figsize=(16, 12))

        ax = axes[0, 0]
        ax.plot(steps, self.last_temps, label='Room')
        ax.plot(steps, self.last_outdoor, label='Outdoor', alpha=0.5)
        ax.axhspan(20, 24, alpha=0.1, color='green', label='Comfort 20-24')
        ax.axhline(22, color='r', linestyle='--', label='Target 22')
        ax.set_title(f'Episode {episode} | Temperature')
        ax.set_xlabel('Step (15min)')
        ax.set_ylabel('°C')
        ax.legend()
        ax.grid(True, alpha=0.3)

        ax = axes[0, 1]
        ax.plot(steps, self.last_rewards, 'tab:orange')
        ax.set_title('Step Rewards')
        ax.set_xlabel('Step')
        ax.set_ylabel('Reward')
        ax.grid(True, alpha=0.3)

        ax = axes[1, 0]
        ax.scatter(steps, self.last_actions, s=12, alpha=0.7)
        ax.set_title('Action Index per Step')
        ax.set_xlabel('Step')
        ax.set_ylabel('Action idx (0-23)')
        ax.set_ylim(-1, max(self.last_actions) + 2)
        ax.grid(True, alpha=0.3)

        ax = axes[1, 1]
        items = sorted(self.last_action_counts.items(), key=lambda x: x[0])
        if items:
            labels, counts = zip(*items)
            ax.bar(labels, counts, color='tab:blue')
        ax.set_title('Action Histogram (Episode)')
        ax.set_xlabel('Action idx')
        ax.set_ylabel('Count')
        ax.grid(True, axis='y', alpha=0.3)

        ax = axes[2, 0]
        ax.plot(steps, supply_seq, label='TSupply (°C)')
        ax.plot(steps, heat_seq, label='Heat set (°C)')
        ax.plot(steps, cool_seq, label='Cool set (°C)')
        ax.set_title('Supply / Setpoints')
        ax.set_xlabel('Step')
        ax.set_ylabel('°C')
        ax.legend()
        ax.grid(True, alpha=0.3)

        ax = axes[2, 1]
        ax.plot(steps, fan_seq, label='Fan u', color='tab:purple')
        ax.set_title('Fan Control')
        ax.set_xlabel('Step')
        ax.set_ylabel('0-1')
        ax.set_ylim(0, 1.05)
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        if save_dir:
            os.makedirs(save_dir, exist_ok=True)
            fig.savefig(os.path.join(save_dir, f"dqn_curves_ep{episode}.png"), dpi=150)
        plt.show()

# =====================================================================
# 训练循环
# =====================================================================

def linear_eps(step):
    return max(EPS_END, EPS_START - (EPS_START - EPS_END) * step / EPS_DECAY_STEPS)


def soft_update(target, online, tau):
    for tp, op in zip(target.parameters(), online.parameters()):
        tp.data.copy_(tau * op.data + (1 - tau) * tp.data)


def train():
    print("=" * 70, flush=True)
    print("🚀 BOPTEST DQN 训练 (修复版)", flush=True)
    print("=" * 70, flush=True)

    log("检查BOPTEST连接...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/testcases", timeout=10)
        log("✅ 连接成功!")
    except Exception as e:
        log(f"❌ 连接失败: {e}", "ERROR")
        log(f"请确认BOPTEST_URL是否正确: {BOPTEST_URL}", "ERROR")
        return None

    env = BOPTESTEnv()
    monitor = Monitor()

    q_net = QNetwork(env.obs_dim, env.action_dim)
    target_net = QNetwork(env.obs_dim, env.action_dim)
    target_net.load_state_dict(q_net.state_dict())
    optimizer = optim.Adam(q_net.parameters(), lr=LR)

    buffer = ReplayBuffer(BUFFER_SIZE, env.obs_dim)

    global_step = 0
    best_eval = -1e9

    for episode in range(1, TOTAL_EPISODES + 1):
        print(f"\n{'=' * 60}", flush=True)
        print(f"📊 Episode {episode}/{TOTAL_EPISODES}", flush=True)
        print(f"{'=' * 60}", flush=True)
        
        try:
            state = env.reset()
        except Exception as e:
            log(f"❌ Reset失败: {e}", "ERROR")
            env.stop()
            time.sleep(2)
            continue

        total_reward = 0.0
        temps = []
        actions_seq = []
        rewards_seq = []
        outdoor_seq = []

        for step in range(STEPS_PER_EPISODE):
            eps = linear_eps(global_step)
            with torch.no_grad():
                q_vals_all = q_net(torch.FloatTensor(state).unsqueeze(0)).squeeze(0)
            
            if np.random.rand() < eps:
                action = np.random.randint(env.action_dim)
            else:
                action = int(q_vals_all.argmax().item())

            next_state, reward, done, info = env.step(action)
            buffer.add(state, action, reward, float(done), next_state)

            state = next_state
            total_reward += reward
            temps.append(info["room_temp"])
            actions_seq.append(action)
            rewards_seq.append(reward)
            outdoor_seq.append(info["outdoor_temp"])
            global_step += 1

            if (step + 1) % 10 == 0:
                top3_q, top3_actions = torch.topk(q_vals_all, min(3, env.action_dim))
                top3_str = ", ".join([f"A{a.item()}:Q={q.item():.2f}" for q, a in zip(top3_q, top3_actions)])
                print(
                    f"  Step {step+1:3d} | "
                    f"室温={info['room_temp']:5.1f}°C | "
                    f"室外={info['outdoor_temp']:5.1f}°C | "
                    f"R={reward:+6.2f} | "
                    f"动作={action:2d} | "
                    f"{action_to_string(action)}",
                    flush=True,
                )
                print(f"           Top3 Q值: [{top3_str}] | eps={eps:.3f}", flush=True)
                rd = info["reward_detail"]
                print(
                    f"           细分: comfort={rd['comfort']:+.2f}, energy={rd['energy']:+.2f}, smooth={rd['smooth']:+.2f}",
                    flush=True,
                )

            if buffer.idx >= MIN_REPLAY_SIZE or buffer.full:
                obs_b, act_b, rew_b, done_b, nxt_b = buffer.sample(BATCH_SIZE)
                with torch.no_grad():
                    target_q = target_net(nxt_b).max(dim=1)[0]
                    target = rew_b + GAMMA * (1 - done_b) * target_q
                q_vals = q_net(obs_b).gather(1, act_b.unsqueeze(1)).squeeze(1)
                loss = nn.functional.mse_loss(q_vals, target)
                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(q_net.parameters(), 1.0)
                optimizer.step()

                if global_step % TARGET_UPDATE == 0:
                    soft_update(target_net, q_net, TAU)

            if done:
                break

        avg_temp = np.mean(temps) if temps else 0
        monitor.train_rewards.append(total_reward)
        monitor.train_temps.append(avg_temp)
        monitor.log_episode_curves(actions_seq, temps, outdoor_seq, rewards_seq)

        action_counts = Counter(actions_seq)
        unique_actions = len(action_counts)
        most_common = sorted(action_counts.items(), key=lambda x: -x[1])[:3]
        
        print(f"\n✅ Episode {episode} 完成", flush=True)
        print(f"   总奖励: {total_reward:.1f}", flush=True)
        print(f"   平均室温: {avg_temp:.1f}°C (目标 20-24°C)", flush=True)
        print(f"   Epsilon: {eps:.3f} | Buffer: {buffer.idx if not buffer.full else buffer.capacity}/{buffer.capacity}", flush=True)
        print(f"   使用了 {unique_actions} 种不同动作", flush=True)
        print(
            f"   最常用动作: {[(a, action_to_string(a), c) for a, c in most_common]}",
            flush=True,
        )

        env.stop()

        if episode % EVAL_FREQUENCY == 0:
            eval_rewards = []
            for _ in range(2):
                s = env.reset()
                ep_r = 0.0
                for _ in range(STEPS_PER_EPISODE):
                    with torch.no_grad():
                        a = int(q_net(torch.FloatTensor(s).unsqueeze(0)).argmax().item())
                    s, r, d, _ = env.step(a)
                    ep_r += r
                    if d:
                        break
                eval_rewards.append(ep_r)
                env.stop()
            avg_eval = float(np.mean(eval_rewards)) if eval_rewards else 0
            monitor.eval_episodes.append(episode)
            monitor.eval_rewards.append(avg_eval)
            print(f"  🔍 Eval avg reward: {avg_eval:.2f}", flush=True)
            if avg_eval > best_eval:
                best_eval = avg_eval
                torch.save(q_net.state_dict(), "best_dqn.pth")
                log(f"💾 保存最佳模型 best_dqn.pth | eval={avg_eval:.2f}")

            monitor.plot()
            monitor.plot_episode_curves(episode, PLOT_SAVE_DIR)

    torch.save(q_net.state_dict(), "final_dqn.pth")
    log("训练结束，已保存 final_dqn.pth")
    monitor.plot_episode_curves("final", PLOT_SAVE_DIR)
    return q_net


print("=" * 60)
print("📌 DQN 代码已加载 (修复版)")
print("动作空间:", NUM_ACTIONS)
print("运行: train()")
print("=" * 60)

📌 DQN 代码已加载 (修复版)
动作空间: 24
运行: train()


In [ ]:
train()

🚀 BOPTEST DQN 训练 (修复版)
[15:48:16] INFO  | 检查BOPTEST连接...
[15:48:16] INFO  | ✅ 连接成功!

📊 Episode 1/500
[15:48:16] INFO  | 开始重置环境...
[15:48:16] INFO  | 选择测试案例: bestest_air
[15:48:16] INFO  | 尝试选择测试案例 (尝试 1/3)...
[15:48:20] INFO  | ✅ Test ID: 8406bbe3-232e-4482-8a43-48f436c23f3c
[15:48:23] INFO  |   → Initialize...
[15:48:24] INFO  |   ✓ Initialize 成功
[15:48:24] INFO  |   → Set step...
[15:48:24] INFO  |   ✓ Set step 成功
[15:48:24] INFO  |   → Advance...
[15:48:24] INFO  | ✅ 环境重置成功
  Step  10 | 室温= 20.4°C | 室外= -0.0°C | R= -1.11 | 动作=20 | Fan=0.9|供风=23°C|热设=21.0°C|冷设=26.0°C
           Top3 Q值: [A14:Q=17.75, A19:Q=8.42, A22:Q=7.07] | eps=0.999
           细分: comfort=-0.37, energy=-0.74, smooth=-0.00
  Step  20 | 室温= 19.4°C | 室外= -0.0°C | R= -3.10 | 动作=14 | Fan=0.6|供风=23°C|热设=23.0°C|冷设=26.0°C
           Top3 Q值: [A14:Q=17.81, A19:Q=8.45, A22:Q=7.06] | eps=0.999
           细分: comfort=-2.43, energy=-0.66, smooth=-0.00
  Step  30 | 室温= 19.2°C | 室外=  2.4°C | R= -3.87 | 动作=13 | Fan=0.6|供风=23°C|热设